In [1]:
import tensorflow
import tensorflow.keras
import pandas

In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
import numpy as np

In [5]:
print("The shape of X_train {0}".format(X_train.shape))
print("The shape of Y_train {0}".format(Y_train.shape))
print("The shape of X_test {0}".format(X_test.shape))
print("The shape of Y_test {0}".format(Y_test.shape))

The shape of X_train (60000, 28, 28)
The shape of Y_train (60000,)
The shape of X_test (10000, 28, 28)
The shape of Y_test (10000,)


In [6]:
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.models import *

In [7]:
X_train, X_test = map(lambda x: np.expand_dims(x,axis=-1), [X_train,X_test])

In [8]:
input_data = Input([28,28,1])
x = Conv2D(128, kernel_size=3,strides=(2,2), activation='relu')(input_data)
x = MaxPooling2D((2,2))(x)
x_1 = Conv2D(128, kernel_size=3, strides=(2,2),activation='tanh')(input_data)
x_1 = MaxPooling2D((2,2))(x_1)
cat_1 = Concatenate(axis=1)([x,x_1])
act_1 = Dense(64,activation='relu')(cat_1)
flat = Flatten()(act_1)
output = Dense(10,activation='softmax')(flat)
model = Model(input_data,output)

In [9]:
class myCallback(Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs=None):
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
                print("\nReached 99% accuracy so cancelling training!")

                # Stop training once the above condition is met
                self.model.stop_training = True

In [10]:
model.compile(
    optimizer=AdamW(learning_rate =0.001),
    loss='sparse_categorical_crossentropy',
    metrics='accuracy',
)

In [11]:
_ = model.fit(X_train,Y_train,epochs=15,callbacks=myCallback())

Epoch 1/15
1875/1875 [==============================] - 15s 5ms/step - loss: 0.2311 - accuracy: 0.9428
Epoch 2/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0966 - accuracy: 0.9714
Epoch 3/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0824 - accuracy: 0.9749
Epoch 4/15
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0728 - accuracy: 0.9778
Epoch 5/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0645 - accuracy: 0.9803
Epoch 6/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0591 - accuracy: 0.9820
Epoch 7/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0530 - accuracy: 0.9837
Epoch 8/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0484 - accuracy: 0.9851
Epoch 9/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0467 - accuracy: 0.9855
Epoch 10/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.04

In [12]:
pred = model.predict(X_test)

313/313 [==============================] - 1s 3ms/step


In [13]:
from sklearn.metrics import roc_auc_score as acc

In [14]:
print("The ROC_AUC_Score is {0:.5f}".format(acc(Y_test,pred,multi_class = 'ovo')))

The ROC_AUC_Score is 0.99959


In [ ]:
model.save("Custom_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
